### 1. ライブラリ・関数の読み込み

In [ ]:

# install.packages(c("Matrix", "MatrixExtra", "recommenderlab", "cmfrec", "ggplot2", "reshape2", "viridis", "gridExtra"))
library(cmfrec)
library(Matrix)
library(MatrixExtra)
library(recommenderlab)
library(cmfrec)
library(ggplot2)
library(reshape2)
library(viridis)
library(gridExtra)
source("cmf.R")


### 2. データの読み込み

In [ ]:
brvehins <- load_bravehins(cas_dataset_path = "/Users/spectee/research/CASdatasets-master/data")
brvehins <- brvehins[grepl("Honda", brvehins$VehModel), ]
claim_types <- c("ClaimAmountRob", "ClaimAmountPartColl", "ClaimAmountTotColl", "ClaimAmountFire", "ClaimAmountOther")
brvehins[, "ClaimTotal"] <- rowSums(brvehins[, claim_types])
str(brvehins)

# write.csv(brvehins, file = "data/brvehins_org.csv", row.names = FALSE)


### 3. データの下処理

In [ ]:
# category_to_analyze <- c("Gender", "DrivAge")
# category_to_analyze <- c("VehGroup", "VehYear")
category_to_analyze <- c("VehModel", "Area")

# 型だけに注目するよりもセル自体に注目した方が良いと思った

premium_total <- get_total(brvehins, category_to_analyze, "PremTotal", 0)
exposure_total <- get_total(brvehins, category_to_analyze, "ExposTotal", 100)
claim_total <- get_total(brvehins, category_to_analyze, "ClaimTotal")

# write.csv(exposure_total, "exposure_total.csv")
# write.csv(claim_total, "claim_total.csv")

# 純率にするか損害率にするかは要相談
# pure_premium <- premium_total / exposure_total
pure_premium <- claim_total / exposure_total
loss_ratio <- claim_total / premium_total

# write.csv(pure_premium, "pure_premium.csv")
# write.csv(loss_ratio, "loss_ratio.csv")


### 4. ハイパーパラメータの最適化

In [ ]:
options(warn = -1)
best_params <- optimize_params(
  X = pure_premium, n_folds = 4, k_values = c(2, 5, 10, 15, 20), lambda_values = c(0.01, 0.1, 1, 10)
)
print(best_params)

### 5. 予測精度の検証

In [ ]:
split <- train_test_split(pure_premium)

# # export split$X_train to csv
# write.csv(split$X_train, "data/split$X_train.csv")
# # export split$X_test to csv
# write.csv(split$X_test, "data/split$X_test.csv")

# cmf_args <- list(X = split$X_train, k = best_params$k, lambda = best_params$lambda, niter = 30, nonneg = TRUE, verbose = FALSE)
# cmf_args <- list(X = split$X_train, k = best_params$k, lambda = best_params$lambda, niter = 30, nonneg = FALSE, verbose = FALSE)
cmf_args <- list(X = split$X_train, k = best_params$k, lambda = best_params$lambda, niter = 30, nonneg = TRUE, verbose = FALSE, center = FALSE)


model <- do.call(CMF, cmf_args)

pred <- get_prediction(model, split$X_test)
calc_rmse(pred, split$X_test)


### 6. 予測結果の可視化

In [ ]:
# テストデータにおける予測値と実測値の散布図

df <- data.frame(as.vector(split$X_test), as.vector(pred))
df <- na.omit(df)
colnames(df) <- c("actual", "prediction")
p <- ggplot(df, aes(x = actual, y = prediction)) +
    geom_point() +
    geom_abline(intercept = 0, slope = 1, color = "red") +
    xlim(0, 2500) +
    ylim(0, 2500) +
    labs(title = "Scatter plot of predicted vs. true values", x = "True values", y = "Predicted values")
p


In [ ]:
options(repr.plot.width = 25, repr.plot.height = 20)

# 全区分に対するヒートマップ（実績）
p1 <- visualize_heatmap(pure_premium)

# 全区分に対するヒートマップ（欠測を含む推定値）
actual <- pure_premium
actual[is.na(actual)] <- 0
estimated <- get_prediction(model, actual)
p2 <- visualize_heatmap(estimated)

grid.arrange(p1, p2, ncol = 2)
